In [5]:
from multiprocessing import Pool
from functools import partial
import os
import numpy as np

from mdp_sequence_reader import MDPSequenceReader
from training_pomc_util import *
from pomc import PartiallyObservableMarkovChain

In [6]:
# configuration
sequence_root_dir = 'sample_sequence'
pomdp_setup_root_dir = 'sample_pomdp_setup'
scenario_name = 'sharp_0_0'

max_em_iterations = 50

min_num_states = 2
max_num_states = 15
model_retries = 6

In [7]:
# read the sequence
reader = MDPSequenceReader(os.path.join(sequence_root_dir, scenario_name + '.txt'))
print("Num states: %d" % reader.get_num_states())
print("Num steps: %d" % reader.get_num_steps())

Num states: 12
Num steps: 10000


In [8]:
# generate environment file
pomdp_setup_folder = os.path.join(pomdp_setup_root_dir, scenario_name)
if not os.path.exists(pomdp_setup_folder):
    os.makedirs(pomdp_setup_folder)
reader.generate_grid_world_setup_file(os.path.join(pomdp_setup_folder, 'world.txt'))

In [9]:
# get the sequences
observations = reader.get_observation_sequence()
actions = reader.get_action_sequence()

In [10]:
# get the initial model
action_space = [0, 1, 2, 3]   # (up, right, down, left)
num_observables = 16   # binary measurements of 4 directions

In [11]:
# model training
def train_model(num_states, round_idx, action_space, num_observables, actions, observations,
                converge_improvement_threshold=2., converge_improve_retry=3):
    
    # initialize model
    init_model = initialize_random_pomc(num_states, action_space, num_observables)
    
    # iterate the model
    model = init_model
    log_likelihood = -1e100  # very small
    best_log_likelihood = log_likelihood
    convergence_count = 0
    for r in range(max_em_iterations):
        alist, c = improve_params(xs=actions, ys=observations, m=model)
        new_model = PartiallyObservableMarkovChain(alist, c, model.init)

        next_log_likelihood = get_log_likelihood(make_tableaus(xs=actions, ys=observations, m=new_model))
        #print("round=%d, log likelihood=%f" % (r+1, next_log_likelihood))

        model = new_model
        log_likelihood = next_log_likelihood
    
        # check convergence condition
        if log_likelihood - best_log_likelihood > converge_improvement_threshold:
            convergence_count = 0
        else:
            convergence_count += 1
            if convergence_count == converge_improve_retry:
                break
        
        best_log_likelihood = max(best_log_likelihood, log_likelihood)
    
    print("num_states=%d, round_idx=%d, log_likelihood=%f" % (num_states, round_idx, log_likelihood))
    return num_states, round_idx,  model, log_likelihood, best_log_likelihood

In [12]:
# generate task info for multi-processing
tasks = []
for i_num_states in range(min_num_states, max_num_states+1):
    for i_round in range(model_retries):
        tasks.append((i_num_states, i_round))

In [13]:
# do it so fast!
partially_fed_train_model_func = partial(
    train_model,
    action_space=action_space,
    num_observables=num_observables,
    actions=actions,
    observations=observations,
)
results = Pool().starmap(partially_fed_train_model_func, tasks)

num_states=3, round_idx=3, log_likelihood=-16270.467059
num_states=3, round_idx=0, log_likelihood=-16270.467059
num_states=5, round_idx=3, log_likelihood=-11682.853936
num_states=5, round_idx=0, log_likelihood=-11682.853936
num_states=4, round_idx=0, log_likelihood=-15138.241465
num_states=4, round_idx=3, log_likelihood=-15138.241465
num_states=2, round_idx=3, log_likelihood=-19757.484683
num_states=2, round_idx=0, log_likelihood=-19757.484683
num_states=5, round_idx=4, log_likelihood=-11638.107111
num_states=3, round_idx=1, log_likelihood=-16698.005212
num_states=3, round_idx=4, log_likelihood=-16698.005212
num_states=5, round_idx=1, log_likelihood=-11638.107111
num_states=4, round_idx=4, log_likelihood=-14722.880264
num_states=4, round_idx=1, log_likelihood=-14722.880264
num_states=2, round_idx=4, log_likelihood=-18448.133443
num_states=2, round_idx=1, log_likelihood=-18448.133443
num_states=3, round_idx=5, log_likelihood=-16891.542949
num_states=3, round_idx=2, log_likelihood=-16891

In [14]:
result_table = [[None for _ in range(model_retries)] for _ in range(max_num_states+1)]
for result in results:
    num_states, round_idx, *rest = result
    result_table[num_states][round_idx] = rest
    
pomc_models_folder = os.path.join(pomdp_setup_folder, 'pomc_models')
if not os.path.exists(pomc_models_folder):
    os.makedirs(pomc_models_folder)
for i_num_states in range(min_num_states, max_num_states+1):
    best_model_idx = np.argmax([logl for _, logl, _ in result_table[i_num_states]])
    best_model, best_final_likelihood, _ = result_table[i_num_states][best_model_idx]
    print("%d states, best likelihood=%f" % (i_num_states, best_final_likelihood))
    best_model.dump(os.path.join(pomc_models_folder, "best_%d_states.txt" % i_num_states))

2 states, best likelihood=-18448.133443
3 states, best likelihood=-16270.467059
4 states, best likelihood=-13818.539399
5 states, best likelihood=-11294.712059
6 states, best likelihood=-8588.247718
7 states, best likelihood=-6850.989348
8 states, best likelihood=-4764.318384
9 states, best likelihood=-3669.664980
10 states, best likelihood=-2511.752892
11 states, best likelihood=-565.877132
12 states, best likelihood=-2.033385
13 states, best likelihood=-572.989864
14 states, best likelihood=-567.335505
15 states, best likelihood=-2.818989
